In [40]:
from wilcoxon.spiderman import *
import re
import pandas as pd
import numpy as np

In [38]:
allStrings(findPreviouses(site.html.find(itemprop="articleBody")("ul")[0],6))

'Algorithms & Theory'

In [113]:
## MODS

site = website("https://www.comp.nus.edu.sg/programmes/ug/focus/")

Ls = []
for i,ul in enumerate(site.html.find(itemprop="articleBody")("ul")):
    
    if i%2 == 0:
        primary = "P"
        focus = allStrings(ul.find_previous("h3"))
    else:
        primary = "E"
        focus = np.nan
            
    for li in ul("li"):
        
        L = {"Code":strippedStrings(li)[0],
            "Name":strippedStrings(li)[-1],
            "Focus":focus,
            "Primary":primary}
        Ls.append(L)
        
mods = pd.DataFrame(Ls)
mods = mods.fillna(method = "pad")

mods.Focus = mods.Focus + " (" + mods.Primary + ")"
mods.Focus = mods.groupby("Code").Focus.transform(lambda x: ", ".join(x))

mods = mods.drop_duplicates("Code")

In [114]:
## REQUIREMENTS

site = website("https://www.comp.nus.edu.sg/cug/catalogue/csmodules/#_CS4275_Programming")

Ls = []

for b in site.html("b"):
    try:
        code = strippedStrings(b)[0].split()[0]
        name = " ".join(strippedStrings(b)[0].split()[1:])
        info = strippedStrings(findNexts(b,2))
        mc = " ".join(info[0].split(":")[1:])
        workload = " ".join(info[1].split(":")[1:])
        req = " ".join(info[2].split(":")[1:])
        clu = " ".join(info[3].split(":")[1:])
        cross = " ".join(info[4].split(":")[1:])
        details = "\n".join(info[5:])
        L = {"Code":code,
            "Name":name,
            "MC":mc,
            "Workload":workload,
            "Requirement":req,
            "Reclusion":clu,
            "Cross":cross,
            "Details":details}
        Ls.append(L)
    except: continue
        
reqs = pd.DataFrame(Ls)

In [115]:
reqs = reqs.merge(mods[["Code","Focus","Primary"]],how="left",on="Code")
reqs = reqs.applymap(lambda x: str(x).strip())
reqs.to_clipboard(index=False)